In [22]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息素材準備
handler執行方法設計
啟動應用


'''

'\n掛載Google硬碟\n安裝套件\n引用套件\nAPP應用準備\n消息素材準備\nhandler執行方法設計\n啟動應用\n\n\n'

In [23]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
'''

流程解析

'''


'\n\n流程解析\n\n'

In [25]:
!pip install line-bot-sdk flask flask-ngrok

In [26]:
'''
引用套件
'''


# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [27]:
'''
建置主程序app

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("fe9W+98abzzGHvjleSYFDLWQYEVgSPeDzwiitTXPNcDZa+oSBnokKSsZ99yDT7HXmlIK64GWNmzGpwTULv79BAPxxttdItA/5UkXsqe0ud53cmzXCpoumhzER+fLdWPKfZUq7deFCj9XwVXiAmrvcwdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler('a95c52182fdf038d8b325a54b939e824')

In [28]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [29]:
from linebot.models.events import FollowEvent

@handler.add(FollowEvent)
def handle_follow_event(event):

    line_bot_api.link_rich_menu_to_user(event.source.user_id,"richmenu-efc259cfef923c18205831afcec975cf")  

In [30]:
'''

消息素材準備

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)


In [31]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
  "@HOME" : "richmenu-efc259cfef923c18205831afcec975cf",
  "@知識補帖" : "richmenu-6bc9cefd1e2069efc38717da563c4aeb",
  "@最新資訊" : "richmenu-66790fa3563d55d65b74701e07da68ff"
}

In [32]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
  line_bot_api.link_rich_menu_to_user(event.source.user_id,template_message_dict.get(event.message.text))

In [ ]:
# 主程序運行
app.run()